In [1]:
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
# train_data = []
# val_data = []

# for folder in os.listdir(data_path):
#     folder_path = os.path.join(data_path, folder)
#     file = os.listdir(folder_path)
#     num_train = int(0.8 * len(file))
#     files_train = random.sample(file, num_train)
#     files_val = list(set(file) - set(files_train))
    
#     for file in files_train:
#         file_path = os.path.join(folder_path, file)
#         img = cv2.imread(file_path)
#         img = cv2.resize(img, (224,224))
#         train_data.append((img, folder))
        
#     for file in files_val:
#         file_path = os.path.join(folder_path, file)
#         img = cv2.imread(file_path)
#         img = cv2.resize(img, (224,224))
#         val_data.append((img, folder))





import os
import random
import cv2

data_path = 'train/'
train_data = []
val_data = []

for folder in os.listdir(data_path):
    # Exclude .DS_Store directory
    if folder == '.DS_Store':
        continue
    
    folder_path = os.path.join(data_path, folder)
    file = os.listdir(folder_path)
    num_train = int(0.6 * len(file))
    files_train = random.sample(file, num_train)
    files_val = list(set(file) - set(files_train))
    
    for file in files_train:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        train_data.append((img, folder))
        
    for file in files_val:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        val_data.append((img, folder))


In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(10, 5))
plt.suptitle('LABELS OF EACH IMAGE')

for (img, label), ax in zip(random.sample(train_data, 8), axes.flatten()):
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.grid(False)
    ax.set_title(label)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB) )

plt.show()

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
base_model.trainable = False

In [ ]:
num_classes = 2
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# train_data = [(preprocess_input(input), label) for input, label in train_data]
# val_data = [(preprocess_input(input), label) for input, label in val_data]

X_train, y_train = zip(*train_data)
X_val, y_val = zip(*val_data)

X_train = preprocess_input(np.array(X_train))
X_val = preprocess_input(np.array(X_val))

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)

In [ ]:
le.classes_

In [ ]:
EPOCHS = 7
BATCH_SIZE = 64
history = model.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot),
                   epochs = EPOCHS, batch_size=BATCH_SIZE)

model.save('pneumonia.h5')

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model)

In [2]:
import os
import cv2
import random
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.optimizers.legacy.Adam import Adam.
data_path = 'train/'
train_data = []
val_data = []

# Adjust the percentage of data used for training
train_percentage = 0.7  # 60% for training

for folder in os.listdir(data_path):
    if folder == '.DS_Store':
        continue
    
    folder_path = os.path.join(data_path, folder)
    file = os.listdir(folder_path)
    
    num_train = int(train_percentage * len(file))
    files_train = random.sample(file, num_train)
    files_val = list(set(file) - set(files_train))
    
    for file in files_train:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))
        train_data.append((img, folder))
        
    for file in files_val:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))
        val_data.append((img, folder))

# Larger pre-trained model for knowledge distillation
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Smaller model for distillation
small_model = MobileNet(weights=None, input_shape=(224, 224, 3), classes=2)
distillation_layer = GlobalAveragePooling2D()(base_model.output)
distillation_layer = Dense(512, activation='relu')(distillation_layer)
distillation_layer = Model(inputs=base_model.input, outputs=distillation_layer)

# Final output layer for small model
predictions = Dense(2, activation='softmax')(small_model.output)

# Combine small model with distillation layer
model = Model(inputs=small_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare data for training
X_train, y_train = zip(*train_data)
X_val, y_val = zip(*val_data)

X_train = np.array(X_train)
X_val = np.array(X_val)

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

y_train_one_hot = to_categorical(y_train_encoded, 2)
y_val_one_hot = to_categorical(y_val_encoded, 2)

# Train the model with knowledge distillation
EPOCHS = 20
BATCH_SIZE = 32
history = model.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot),
                    epochs=EPOCHS, batch_size=BATCH_SIZE)

# Save the trained model
model.save('small_pneumonia_model3.h5')


Epoch 1/20


2024-02-07 12:55:07.102089: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


115/115 [==============================] - 26s 210ms/step - loss: 0.6074 - accuracy: 0.8781 - val_loss: 0.6114 - val_accuracy: 0.7427
Epoch 2/20
115/115 [==============================] - 22s 186ms/step - loss: 0.5005 - accuracy: 0.9104 - val_loss: 0.5776 - val_accuracy: 0.7427
Epoch 3/20
115/115 [==============================] - 21s 185ms/step - loss: 0.4334 - accuracy: 0.9145 - val_loss: 0.5703 - val_accuracy: 0.7427
Epoch 4/20
115/115 [==============================] - 21s 187ms/step - loss: 0.3957 - accuracy: 0.9014 - val_loss: 0.5772 - val_accuracy: 0.7427
Epoch 5/20
115/115 [==============================] - 21s 186ms/step - loss: 0.3496 - accuracy: 0.9197 - val_loss: 0.5918 - val_accuracy: 0.7427
Epoch 6/20
115/115 [==============================] - 22s 188ms/step - loss: 0.3270 - accuracy: 0.9181 - val_loss: 0.5586 - val_accuracy: 0.7688
Epoch 7/20
115/115 [==============================] - 21s 186ms/step - loss: 0.2920 - accuracy: 0.9342 - val_loss: 0.6871 - val_accuracy: 0.5

In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load the saved model
loaded_model = load_model('small_pneumonia_model3.h5')

# Load and preprocess the image
image_path = 'person1_bacteria_2.jpeg'
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))  # Resize to match the input size of the model
image = np.expand_dims(image, axis=0)  # Add batch dimension
image = preprocess_input(image)  # Preprocess the image

# Make predictions
predictions = loaded_model.predict(image)

# Get the predicted class
predicted_class = np.argmax(predictions, axis=1)

# Print the predicted class
print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 303ms/step
Predicted class: [1]
